# Top Foods Analysis

This notebook analyzes top foods and feature importances from trained Random Forest models. Configure the model/window and paths in the first code cell (Cell 2). It produces top-food rankings, feature importance plots, and distributions of top food features.

In [ ]:
# Cell 2: Configuration and imports
import json
from pathlib import Path
import joblib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Config - adjust these paths to the model/window you want to analyze
MODEL_WINDOW_DIR = Path('models/rf_out_pain/window_4h')  # example
FEATURES_JSON = MODEL_WINDOW_DIR / 'features.json'
IMPORTANCES_CSV = MODEL_WINDOW_DIR / 'feature_importances.csv'
ML_PARQUET = Path('scripts/data/curated/foods_ml.parquet')
MEALS_PARQUET = Path('scripts/data/raw/meals_with_symptoms.parquet')
TOP_N = 10

plt.style.use('seaborn-v0_8')

# helper to robustly get positive-class probabilities

def robust_pos_probs(pipe, X):
    proba = pipe.predict_proba(X)
    if proba.ndim == 2 and proba.shape[1] == 1:
        model = pipe.named_steps.get('model')
        classes = getattr(model, 'classes_', None)
        if classes is not None and len(classes) == 1 and classes[0] == 1:
            return proba[:, 0]
        return np.zeros(proba.shape[0])
    model = pipe.named_steps.get('model')
    classes = getattr(model, 'classes_', None)
    if classes is not None and 1 in classes:
        pos_idx = int(np.where(classes == 1)[0][0])
    else:
        pos_idx = proba.shape[1] - 1
    return proba[:, pos_idx]

In [ ]:
# Cell 3: Load model, data, compute predictions, and aggregate top foods

# load model artifacts
model_pkl = MODEL_WINDOW_DIR / 'model.pkl'
pipe = joblib.load(model_pkl)
feat_cols = json.load(open(FEATURES_JSON))

# load data
ml_df = pd.read_parquet(ML_PARQUET)
meals_df = pd.read_parquet(MEALS_PARQUET)

# merge meals with ML features
merged = meals_df.merge(ml_df, on='canonical', how='left')
# ensure meal_datetime parsing
if 'meal_datetime' not in merged.columns:
    merged['meal_datetime'] = pd.to_datetime(merged.get('meal_datetime') if 'meal_datetime' in merged else merged.get('datetime', merged.get('date')))

# ensure features present
for c in feat_cols:
    if c not in merged.columns:
        merged[c] = 0.0
X = merged[feat_cols].fillna(0.0)
probs = robust_pos_probs(pipe, X)
merged['pred_prob'] = probs

# aggregate by canonical food
agg = merged.groupby('canonical').agg(n_meals=('pred_prob', 'size'), mean_pred=('pred_prob', 'mean')).reset_index()
agg = agg.sort_values('mean_pred', ascending=False)

# save
out_csv = MODEL_WINDOW_DIR / f'top_foods_{MODEL_WINDOW_DIR.name}.csv'
agg.to_csv(out_csv, index=False)
print('Wrote', out_csv)

# show top N
agg.head(TOP_N)

In [ ]:
# Cell 4: Load feature importances and visualize top nutrient features

if IMPORTANCES_CSV.exists():
    fi = pd.read_csv(IMPORTANCES_CSV)
    # keep only features present in features.json order
    fi = fi[fi['feature'].isin(feat_cols)].copy()
    fi = fi.sort_values('importance', ascending=False)
else:
    # if no importances CSV (e.g., single-class or missing), synthesize from model if possible
    model = pipe.named_steps.get('model')
    if hasattr(model, 'feature_importances_'):
        fi = pd.DataFrame({'feature': feat_cols, 'importance': model.feature_importances_})
        fi = fi.sort_values('importance', ascending=False)
    else:
        fi = pd.DataFrame({'feature': feat_cols, 'importance': [0.0]*len(feat_cols)})

# top N features
top_feats = fi.head(TOP_N)

# bar plot of top features
plt.figure(figsize=(8, TOP_N * 0.5 + 1))
sns.barplot(x='importance', y='feature', data=top_feats, palette='viridis')
plt.title('Top feature importances')
plt.tight_layout()
plt.show()

In [ ]:
# Cell 5: Visualize top foods (bar chart) and nutrient scatter

# merge top N with ML features for plotting
topN = agg.head(TOP_N).merge(ml_df, on='canonical', how='left')

# horizontal bar chart of top N foods
plt.figure(figsize=(8, TOP_N * 0.5 + 1))
sns.barplot(x='mean_pred', y='canonical', data=topN, palette='magma')
plt.xlabel('Mean predicted risk')
plt.ylabel('Food (canonical)')
plt.title(f'Top {TOP_N} foods by predicted risk')
plt.tight_layout()
plt.show()

# nutrient scatter: calories vs protein colored by mean_pred
if 'calories' in topN.columns and 'protein_g' in topN.columns:
    plt.figure(figsize=(6, 5))
    sc = plt.scatter(topN['calories'], topN['protein_g'], c=topN['mean_pred'], cmap='coolwarm', s=80)
    for i, txt in enumerate(topN['canonical']):
        plt.annotate(txt, (topN['calories'].iat[i], topN['protein_g'].iat[i]), textcoords='offset points', xytext=(3,3), fontsize=8)
    plt.colorbar(sc, label='mean_pred')
    plt.xlabel('Calories')
    plt.ylabel('Protein (g)')
    plt.title('Calories vs Protein for Top Foods (colored by mean predicted risk)')
    plt.tight_layout()
    plt.show()
else:
    print('Calories/protein not present in ML dataframe; skipping scatter')

In [ ]:
# Cell 6: Visualize distribution of top food features

# For top K features that look like nutrients (e.g., calories, protein_g), plot distributions
nutrient_like = [f for f in top_feats['feature'].tolist() if f in ml_df.columns]
if not nutrient_like:
    # fallback: take a few base nutrient columns if present
    nutrient_like = [c for c in ['calories','protein_g','fat_g','carbs_g','fiber_g','sugar_g'] if c in ml_df.columns][:TOP_N]

for col in nutrient_like:
    plt.figure(figsize=(6,3))
    sns.boxplot(x=merged[col].dropna())
    plt.title(f'Distribution of {col} across meals')
    plt.tight_layout()
    plt.show()

    plt.figure(figsize=(6,3))
    sns.histplot(merged[col].dropna(), kde=True)
    plt.title(f'Histogram of {col}')
    plt.tight_layout()
    plt.show()